# Токенизация текстов

**Ссылка**, на источник текста.

In [2]:
DATA_URL = "http://az.lib.ru/t/tolstoj_a_k/text_0180.shtml"

Нам понадобятся следующие библиотеки: **PyYAML**, **RNNMorph**, а также **NLTK**

In [1]:
! pip install -q PyYaml==5.3.1
! pip install -q rnnmorph==0.4.0
! pip install -q nltk==3.2.5

     |████████████████████████████████| 276kB 4.0MB/s 
     |████████████████████████████████| 10.5MB 3.9MB/s 
     |████████████████████████████████| 61kB 7.2MB/s 
     |████████████████████████████████| 8.2MB 45.7MB/s 


Создаём объект морфологического анализатора **RNNMorph**.



In [4]:
%tensorflow_version 1.x
import warnings
warnings.filterwarnings('ignore')
from rnnmorph.predictor import RNNMorphPredictor
predictor = RNNMorphPredictor(language="ru")

Скачиваем текст, по которому будем производить дальнейший анализ, с помощью **urllib**.

In [6]:
import urllib.request

opener = urllib.request.URLopener({})
resource = opener.open(DATA_URL)
raw_text = resource.read().decode(resource.headers.get_content_charset()) 

In [7]:
raw_text[:200]

'<html>\r\n<head>\r\n<title>Lib.ru/Классика: Толстой Алексей Константинович. Семья вурдалака</title>\r\n</head>\r\n\r\n<body>\r\n\r\n\r\n<center>\r\n\r\n<h2><a href=/t/tolstoj_a_k/>Толстой Алексей Константинович</a><br>\r\n'

Загруженный текст содержит в себе HTML-теги, от которых нужно избавиться. Для этого, воспользуемся **BeautifulSoup**.

In [9]:
from bs4 import BeautifulSoup

soup = BeautifulSoup(raw_text, features="html.parser")
for script in soup(["script", "style"]):
    script.extract()
cleaned_text = soup.get_text()

In [10]:
cleaned_text[:200]

'\n\nLib.ru/Классика: Толстой Алексей Константинович. Семья вурдалака\n\n\n\nТолстой Алексей Константинович\r\nСемья вурдалака\n\n\nLib.ru/Классика:\n\r\n\n\n[Регистрация]\n \n\r\n\r\n\r\n[Найти] \r\n[Рейтинги]\r\n[Обсуждения]\r\n['

После удаление HTML-тегов, в тексте всё еще осталось множество специальных символов, но они не помешают легко токенизировать текст. С помощью библиотеки **NLTK** разбиваем текст на предложения и слова (**токены**).

In [11]:
from nltk.tokenize import sent_tokenize, word_tokenize
import nltk
nltk.download('punkt')

tokenized_sentences = [word_tokenize(sentence) for sentence in sent_tokenize(cleaned_text)]
"A total of %d 'sentences'" % len(tokenized_sentences)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


"A total of 577 'sentences'"

С помощью метода **str.isalpha()** из стандартной библиотеки **Python** выделим только буквенные токены.

In [28]:
from tqdm import tqdm

predictions = [[pred.normal_form for pred in sent if pred.normal_form.isalpha()] 
               for sent in tqdm(predictor.predict_sentences(sentences=tokenized_sentences), "sentences") ]
predictions[-11:-10]




sentences: 100%|██████████| 577/577 [00:00<00:00, 43203.97it/s]


[['не',
  'помнить',
  'что',
  'произойти',
  'ещё',
  'но',
  'когда',
  'я',
  'прийти',
  'в',
  'себя',
  'быть',
  'уже',
  'вполне',
  'светлый',
  'я',
  'лежать',
  'на',
  'дорога',
  'а',
  'рядом',
  'издыхать',
  'мой',
  'конь']]

Должны получиться следующие значения:

*   Предложений: **577~**
*   Токенов: **8621~** 

In [29]:
len(predictions)

577

In [30]:
non_uniq_tokens = [word for sentence in predictions for word in sentence]
len(non_uniq_tokens) 

8621

Используя **non_uniq_tokens**, стоп-слова для русского языка из библиотеки **nltk.corpus.stopwords** и **nltk.FreqDist**, вычислим, какую долю среди 100 самых частотных токенов в произведении составляют токены, которых нет в списке стоп-слов **nltk.corpus.stopwords**. 

Если на **100** наиболее часто-встречаемых слов, приходится **25**, которые входят в стоп-лист, значит доля значимых слов составит **0.75**. 

In [31]:
import nltk
from nltk import FreqDist
from nltk.corpus import stopwords
nltk.download("stopwords")
STOPWORDS = set(stopwords.words("russian"))

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


['и', 'в', 'во', 'не', 'что']

Должно получиться **0.49~** (допустимая погрешность **0.1**)

In [86]:
no_stops = [token for token in non_uniq_tokens if not token in stopwords.words("russian")]
print("Доля значимых слов: %.2f" %((len(non_uniq_tokens)-len(no_stops))/len(non_uniq_tokens)))

Доля значимых слов: 0.44


Вычислим, сколько токенов встречается в тексте **строго больше** 50 раз.

In [81]:
uniq_tokens = dict(FreqDist(samples=no_stops))
greater_than_fifty = [i for i in uniq_tokens.items() if i[-1] > 50]
print(len(greater_than_fifty))

5


Проверьте себя: должно получиться значение 22 (возможно небольшое расхождение)
